<a href="https://colab.research.google.com/github/ashen-pabasara/fabric-defect-detection/blob/main/prediction_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install & import libraries
!pip install -q huggingface_hub ultralytics gradio

import gradio as gr
from ultralytics import YOLO
import os
import shutil
import glob
import time
from huggingface_hub import hf_hub_download


# Close previous run
try:
    demo.close()
    print("Previous run closed.")
except:
    pass


# Setup model from Hugging Face
REPO_ID = "ashen97/fabric-defect-yolo"
MODEL_FILES = {
    "YOLOv8n": "YOLOv8n.pt",
    "YOLOv8s": "YOLOv8s.pt",
    "YOLOv11": "YOLOv11.pt"
}


# Verify models
print("Checking models...")
MODEL_PATHS = {}

try:
    for display_name, filename in MODEL_FILES.items():
        cached_path = hf_hub_download(repo_id=REPO_ID, filename=filename)
        MODEL_PATHS[display_name] = cached_path
        print(f"✅ Loaded: {display_name}")
except Exception as e:
    print(f"⚠️ Note: Could not verify Hugging Face models ({e}).")


# Prediction function
def detect_defects(input_video, model_name, conf_level):
    try:
        if input_video is None:
            raise gr.Error("Please upload a video first!")

        # Create output folder
        timestamp = int(time.time())
        output_dir = f"/content/gradio_results_{timestamp}"

        # Load Model
        if model_name not in MODEL_PATHS:
             local_path = f"{model_name.split(' ')[0].lower()}.pt"
             if os.path.exists(local_path):
                 model_path = local_path
             else:
                 raise gr.Error(f"Model file for {model_name} not found!")
        else:
             model_path = MODEL_PATHS[model_name]

        model = YOLO(model_path)

        # Run Prediction
        print(f"Processing video with {model_name}...")
        results = model.predict(
            source = input_video,
            save = True,
            conf = conf_level,
            project = output_dir,
            name = "run",
            verbose = False
        )

        # Find Output Video
        save_dir = results[0].save_dir
        files = os.listdir(save_dir)

        # Verify video for .avi or .mp4
        video_file = next((f for f in files if f.endswith(('.avi', '.mp4'))), None)

        if not video_file:
            raise gr.Error("YOLO finished but no video file was saved.")

        full_video_path = os.path.join(save_dir, video_file)

        # Convert to .mp4
        output_mp4 = full_video_path.replace(".avi", "_converted.mp4")

        if not os.path.exists(output_mp4):
            os.system(f"ffmpeg -y -loglevel panic -i '{full_video_path}' -vcodec libx264 '{output_mp4}'")

        return output_mp4

    # Show error in UI
    except Exception as e:
        print(f"Error Details: {str(e)}")
        raise gr.Error(f"System Error: {str(e)}")


# Build UI
inputs = [
    gr.Video(label="Input Video"),
    gr.Dropdown(choices=list(MODEL_FILES.keys()), value=list(MODEL_FILES.keys())[0], label="Model Variant"),
    gr.Slider(0.0, 1.0, value=0.25, step=0.05, label="Confidence Threshold")
]

# Run UI
demo = gr.Interface(
    fn = detect_defects,
    inputs = inputs,
    outputs = gr.Video(label="Predicted Output"),
    title = "Fabric Defect Detection System",
    description = "Upload a video to detect defects in real-time.",
    theme = "soft"
)

demo.launch(share=True, debug=True)


Closing server running on port: 7860
Previous instance closed.
⬇️ Checking models...
✅ Loaded: YOLOv8n
✅ Loaded: YOLOv8s
✅ Loaded: YOLOv11
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://03f525410c48d06e23.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Processing video with YOLOv8n...
WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to /content/gradio_results_1767672273/run
Created dataset file at: .gradio/flagged/dataset1.csv
Processing video with YOLOv8s...
WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=